# 线性回归的简洁实现

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
import numpy as np
import torch
from IPython import display
import random


num_input = 2
num_example = 1000
true_w = [2, -3.4]
true_b = 4.2
# torch.from_numpy(np.random.normal(0, 1, (num_example, num_input))) 意思是生成一个  (num_example, num_input) 指定生成数组的形状，即生成一个大小为 (num_example, num_input) 的矩阵，矩阵中的每个元素都是独立抽样得到的符合 N (0, 1) 分布的值
features = torch.from_numpy(np.random.normal(0, 1, (num_example, num_input)))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
# 添加噪声
labels += torch.from_numpy(np.random.normal(0, 0.01, size=labels.size()))
# features

# 读取数据

In [5]:
import torch.utils.data as Data
from torch import nn

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小样本
data_iter = Data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [31]:
for X, y in data_iter:
    print(X,'\n', y)
    break

tensor([[-0.9471,  1.0378],
        [-1.1339,  0.1654],
        [ 0.4611,  1.1529],
        [-1.4478, -0.6733],
        [-0.3608, -0.0716],
        [-0.0559,  2.2544],
        [-0.3233, -0.6429],
        [-1.3127, -0.3244],
        [ 0.1497,  1.4442],
        [-0.2571, -0.3098]], dtype=torch.float64) 
 tensor([-1.2340,  1.3799,  1.2081,  3.5815,  3.7195, -3.5926,  5.7389,  2.6871,
        -0.3928,  4.7445], dtype=torch.float64)


# 定义模型

In [42]:
class LinearNet(nn.Module):
    def __init__(self, n_features):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(in_features=n_features, out_features=1)
    # forward 向前传播
    def forward(self, x):
        y = self.linear(x)
        return y

# net = LinearNet(num_input)
# print(net)

In [66]:
net = nn.Sequential(
    nn.Linear(num_input, 1, dtype=torch.double)
)
print(net)
print(net[0].weight)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Parameter containing:
tensor([[-0.1750,  0.4688]], dtype=torch.float64, requires_grad=True)


In [57]:
# 可以通过 net.parameters() 来查看模型所有的可学习参数, 此函数将返回一个生成器
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.6731, -0.6546]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.7008], dtype=torch.float64, requires_grad=True)


# 初始化模型参数
## 我们通过 init.normal_ 将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。

In [58]:
from torch.nn import init
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)

Parameter containing:
tensor([0.], dtype=torch.float64, requires_grad=True)

In [59]:
# 定义损失函数
loss = nn.MSELoss() # 均方误差损失 0.5 * (y_hat - y) ** 2

In [60]:
# 定义优化算法
optimizer = torch.optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


# 训练模型

In [62]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1)) # 计算损失
        optimizer.zero_grad() # 梯度清零
        l.backward() # 反向传播
        optimizer.step()
    print('epoch: %d, loss: %f' % (epoch, l.item()))

epoch: 1, loss: 0.000088
epoch: 2, loss: 0.000159
epoch: 3, loss: 0.000157
epoch: 4, loss: 0.000060
epoch: 5, loss: 0.000077
epoch: 6, loss: 0.000057
epoch: 7, loss: 0.000085
epoch: 8, loss: 0.000134
epoch: 9, loss: 0.000052
epoch: 10, loss: 0.000089


In [63]:
dense = net[0]

In [64]:
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9996, -3.4004]], dtype=torch.float64, requires_grad=True)
4.2 Parameter containing:
tensor([4.2003], dtype=torch.float64, requires_grad=True)
